In [ ]:
using CSV, DataFrames, Plots, StatsPlots, Statistics

In [ ]:
directory = "nmar_outliers/"
filelist = [f for f in readdir(directory) if endswith(f, ".csv")]
res = similar(DataFrame(CSV.read(directory*filelist[1])),0)
for i in 1:length(filelist)
    res = vcat(res, DataFrame(CSV.read(directory*filelist[i])))
end
res[!,:method_cat] = map(t -> startswith(t,"Imp-then-Reg") ? "Imp-then-Reg" : t, res[:,:method]);

In [ ]:
res = res[.!isnan.(res[:,:osr2]), :]

In [ ]:
@show length(unique(res[:,:dataset]))

In [ ]:
dataset_list = [d for d in readdir("../datasets/") if length(d) > 0 && d != ".DS_Store"]
setdiff(dataset_list, unique(res[:,:dataset]))

In [ ]:
intersect(dataset_list, unique(res[:,:dataset]))

In [ ]:
setdiff(    filter(t -> t[:method] == "Imp-then-Reg 1", res)[:, :dataset],
            filter(t -> t[:method] == "Affine", res)[:, :dataset])

## Claim 1: Mode-imputation does not help


In [ ]:
#Select only datasets where at least one missing categorical feature
pattern_count_all = DataFrame(CSV.read("pattern_counts_allfeat.csv"))[:,[:Name,:n,:p,:p_miss]]
rename!(pattern_count_all, :p_miss => :p_miss_all)
pattern_count_num = DataFrame(CSV.read("pattern_counts_numonly.csv"))[:,[:Name,:p_miss]]
rename!(pattern_count_num, :p_miss => :p_miss_num)
relevant_data = join(pattern_count_all, pattern_count_num, on=:Name)
filter!(t -> t[:p_miss_all] > t[:p_miss_num], relevant_data)
relevant_data[!,:p_miss_cat] = relevant_data[:,:p_miss_all] .- relevant_data[:,:p_miss_num]
relevant_data

In [ ]:
mi_res = deepcopy(res)
# @show length(unique(mi_res[:,:dataset]))
filter!( t -> t[:dataset] ∈ unique(relevant_data[:,:Name]), mi_res)
# @show length(unique(mi_res[:,:dataset]))
save = unique(mi_res[:,:dataset])
filter!(t -> t[:method] ∈ ["Imp-then-Reg 4", "Imp-then-Reg 5"], mi_res)
# @show length(unique(mi_res[:,:dataset]))
# @show setdiff(save, unique(mi_res[:,:dataset]))
select!(mi_res, Not([:method_cat]))
mi_res[!,:method] = map( t -> t == "Imp-then-Reg 4" ? "Missing_Indicator" : "Mode_Impute", mi_res[:,:method]);
# mi_res[mi_res[:,:osr2] .< 0,:osr2] .= 0.

In [ ]:
setdiff(intersect(relevant_data[:,:Name], res[:,:dataset]), unique(mi_res[:,:dataset]))

In [ ]:
setdiff(    filter(t -> t[:method] == "Imp-then-Reg 4", res)[:, :dataset],
            filter(t -> t[:method] == "Imp-then-Reg 5", res)[:, :dataset])

In [ ]:
#filter!(row -> row[:k] == 5 && row[:kMissing] == 1, mi_res)

In [ ]:
mi_wide = unstack(mi_res, [:dataset, :splitnum, :k, :kMissing], :method, :osr2)
dropmissing!(mi_wide); #TO DO: Check for which dataset/splits this is needed. Should not be 

In [ ]:
names(mi_wide)

In [ ]:
mi_wide[!, :wins] = 1.0 .* (mi_wide[:,:Missing_Indicator] .> mi_wide[:,:Mode_Impute])
mi_wide[!, :improvement] = mi_wide[:,:Missing_Indicator] .- mi_wide[:,:Mode_Impute] 
mi_wide[!, :rel_improvement] = mi_wide[:, :improvement] ./ abs.(mi_wide[:,:Mode_Impute]);

In [ ]:
#Question: Should we average results over each data set before counting improvement?
select!(mi_wide, Not(:splitnum))
mi_wide = aggregate(mi_wide, [:dataset, :k, :kMissing], mean)
rename!(mi_wide, 
        [(:Missing_Indicator_mean => :Missing_Indicator), (:Mode_Impute_mean => :Mode_Impute),
        (:wins_mean => :wins_splitlevel),
        (:improvement_mean => :improvement_splitlevel), (:rel_improvement_mean => :rel_improvement_splitlevel)]
)

mi_wide[!, :wins] = 1.0 .* (mi_wide[:,:Missing_Indicator] .> mi_wide[:,:Mode_Impute])
mi_wide[!, :improvement] = mi_wide[:,:Missing_Indicator] .- mi_wide[:,:Mode_Impute] 
mi_wide[!, :rel_improvement] = mi_wide[:, :improvement] ./ abs.(mi_wide[:,:Mode_Impute]);

In [ ]:
@show mean(mi_wide[:,:wins_splitlevel])
@show mean(mi_wide[:,:wins])
@show sum(mi_wide[:,:wins])

In [ ]:
function iqr(l)
    return quantile(l, .75), quantile(l, .25)
end

In [ ]:
mi_wide[isnan.(mi_wide[:,:improvement]),:]

In [ ]:
res[isnan.(res[:,:osr2]),:]

In [ ]:
metric = :rel_improvement
μ = median(mi_wide[mi_wide[:,:improvement] .> 0,metric])
CI = std(mi_wide[mi_wide[:,:improvement] .> 0,metric]) / sqrt(sum(mi_wide[:,:improvement] .> 0))
IQR = iqr(mi_wide[mi_wide[:,:improvement] .> 0,metric])
@show μ, CI, IQR

In [ ]:
μ = -median(mi_wide[mi_wide[:,:improvement] .< 0,metric])
CI = std(mi_wide[mi_wide[:,:improvement] .< 0,metric]) / sqrt(sum(mi_wide[:,:improvement] .< 0))
IQR = iqr(mi_wide[mi_wide[:,:improvement] .< 0,metric])
@show μ, CI, IQR

In [ ]:
μ = mean(mi_wide[!,metric])
CI = std(mi_wide[!,metric]) / sqrt(nrow(mi_wide))
IQR = iqr(mi_wide[!,metric])
@show μ, CI, IQR

In [ ]:
mi_wide[:,:wins_splitlevel]

In [ ]:
mi_augmented = join(mi_wide, rename(relevant_data, :Name => :dataset), on=:dataset)
mi_augmented[!,:prop_miss_cat] = 1.0 .* mi_augmented[:,:p_miss_cat] ./ mi_augmented[:,:p_miss_all]
mi_augmented[!,:prop_miss_cat2] = 1.0 .* mi_augmented[:,:p_miss_cat] ./ mi_augmented[:,:p]
mi_augmented[!,:wins] = map( t -> (t > 0) ? :blue : :red, mi_augmented[:,:wins]);

In [ ]:
@df mi_augmented scatter(:prop_miss_cat2, :improvement, color=:wins, legend=:none)

In [ ]:
mi_augmented[:,[:dataset,:Mode_Impute, :Missing_Indicator,:rel_improvement]]

## Claim 2: Mean-imputation is good


In [ ]:
#Select only datasets where at least one missing numerical feature
pattern_count_num = DataFrame(CSV.read("pattern_counts_numonly.csv"))[:,[:Name,:n,:p,:p_miss]]
rename!(pattern_count_num, :p_miss => :p_miss_num)
filter!(t -> t[:p_miss_num] > 0, pattern_count_num)

In [ ]:
for d in unique(res[:,:dataset])
    aux = filter(t-> t[:dataset] == d, res)
    filter!(t-> t[:method_cat] == "Imp-then-Reg", aux)
    if size(aux,1) == 0
        @show d
    end
end

In [ ]:
itr_res = deepcopy(res)
@show length(unique(itr_res[:,:dataset]))
filter!(t -> t[:dataset] ∈ unique(pattern_count_num[:,:Name]), itr_res) #filter datasets
@show length(unique(itr_res[:,:dataset]))
filter!(t -> t[:method_cat] == "Imp-then-Reg", itr_res) #compare only impute-then-regress
@show length(unique(itr_res[:,:dataset]))
filter!(t -> t[:method] != "Imp-then-Reg 5", itr_res) #remove mode impute
select!(itr_res, Not(:method_cat))

select!(itr_res, Not(:splitnum))
itr_res = aggregate(itr_res, [:dataset, :method, :k, :kMissing], mean)
@show length(unique(itr_res[:,:dataset]))
rename!(itr_res, :osr2_mean => :osr2)

In [ ]:
itr_res[!,:osr2] .= round.(itr_res[:,:osr2], digits=4)
itr_best = by(itr_res, [:dataset, :k, :kMissing]) do df
    DataFrame(results = [sort(unique(df[:,:osr2]), rev=true)])
end
@show length(unique(itr_best[:,:dataset]))

In [ ]:
itr_res = join(itr_res, itr_best, on = [:dataset, :k, :kMissing])
@show length(unique(itr_res[:,:dataset]))

rank = zeros(size(itr_res,1))
for i in 1:size(itr_res,1)
    rank[i] = findfirst(itr_res[i,:osr2] .== itr_res[i,:results])
end
itr_res[!,:rank] = rank
itr_res[!,:wins] = 1 .* (itr_res[:,:rank] .== 1)
select!(itr_res, Not(:results))
itr_res
@show length(unique(itr_res[:,:dataset]))

In [ ]:
aggregate(itr_res[:,[:method,:wins]], :method, sum)

In [ ]:
aggregate(itr_res[:,[:method,:rank]], :method, t -> sum(t.>0))

In [ ]:
itr_res

In [ ]:
pairwise_wins = unstack(itr_res, [:dataset, :k, :kMissing], :method, :osr2)
methods = Symbol.(["Imp-then-Reg 1", "Imp-then-Reg 2", "Imp-then-Reg 3", "Imp-then-Reg 4"])
for (i, m1) in enumerate(methods), (j, m2) in enumerate(methods)
    if m1 != m2
        pairwise_wins[!, Symbol("V$(i)_beats_V$(j)")] = pairwise_wins[!, m1] .> pairwise_wins[!, m2]
    end
end
dropmissing!(pairwise_wins)
desc = DataFrames.describe(pairwise_wins)
filter!(row -> row[:variable] != :dataset, desc)
desc[!, :sum] = nrow(pairwise_wins) * desc[!, :mean]
@show nrow(pairwise_wins)
select(desc, [:variable, :sum])

In [ ]:
desc

In [ ]:
for m1 in unique(itr_res[:,:method])
    aux1 = filter(t-> t[:method] == m1, itr_res)
    for m2 in unique(itr_res[:,:method])
        if m1 != m2
            aux2 = filter(t-> t[:method] == m2, itr_res)
            wincount = 0 
            for d in unique(itr_res[:,:dataset])
                if aux1[findfirst(aux1[:,:dataset] .== d),:rank] < aux2[findfirst(aux2[:,:dataset] .== d),:rank]
                    wincount += 1
                end
            end 
            println("$m1 wins over $m2 in $wincount instances")
        end
    end
    println()
end

In [ ]:
using StatsBase

In [ ]:
stack(itr_res[:,[:method, :rank]], :method, t-> sum(t .> 0))

In [ ]:
distr_rank = by(itr_res, :method) do df
    [sum(df[:,:rank].==1) (sum(df[:,:rank].==2)) (sum(df[:,:rank].==3)) (sum(df[:,:rank].==4))]
end

groupedbar(Matrix(distr_rank[:,2:end])', bar_position = :dodge, fill=.7, label=["V1" "V2" "V3" "V4"], size= (800,400))
xaxis!("Rank")
yaxis!("Count")
# savefig("nmar_outliers_imputethenreg_freq.png")

In [ ]:
union(unique(itr_res[:,:dataset]), unique(mi_wide[:,:dataset]))

## Claim 3: Compare our approach to state-of-the-art

In [ ]:
#Select only datasets where at least one missing numerical feature
pattern_count_num = DataFrame(CSV.read("pattern_counts_numonly.csv"))[:,[:Name,:p_miss]]
rename!(pattern_count_num, :p_miss => :p_miss_num)
filter!(t -> t[:p_miss_num] > 0, pattern_count_num)

In [ ]:
all_res = deepcopy(res)
filter!(t -> t[:dataset] ∈ unique(pattern_count_num[:,:Name]), all_res) #filter datasets
select!(all_res, Not(:method_cat))
sort!(all_res, [:dataset, :splitnum, :method])

#Average over splits
all_res = aggregate(all_res[:,[:dataset, :method, :osr2]], [:dataset, :method], mean)
rename!(all_res, :osr2_mean => :osr2)
all_res[!,:osr2] = round.(all_res[:,:osr2], digits=4);

best_pred = by(all_res, [:dataset]) do df
    DataFrame(results = [sort(unique(df[:,:osr2]), rev=true)])
end

all_res = join(all_res, best_pred, on = [:dataset])

rank = zeros(size(all_res,1))
for i in 1:size(all_res,1)
    rank[i] = findfirst(all_res[i,:osr2] .== all_res[i,:results])
end
all_res[!,:rank] = rank
select!(all_res, Not(:results))
all_res

In [ ]:
for m1 in unique(all_res[:,:method])
    aux1 = filter(t-> t[:method] == m1, all_res)
    if m1 ∉ ["Imp-then-Reg $i" for i in [1,2,3,5]]
        for m2 in unique(all_res[:,:method])
            if m1 != m2 && m2 ∉ ["Imp-then-Reg $i" for i in [1,2,3,5]]
                aux2 = filter(t-> t[:method] == m2, all_res)
                wincount = 0 
                for d in unique(all_res[:,:dataset])
                    try
                        if aux1[findfirst(aux1[:,:dataset] .== d),:rank] < aux2[findfirst(aux2[:,:dataset] .== d),:rank]
                            wincount += 1
                        end
                    catch
                        ()
                    end
                end 
                println("$m1 wins over $m2 in $wincount instances")
            end
        end
        println()
    end
end